In [ ]:
import os
import yaml
import pypsa

import tz_pypsa as tza
from tz_pypsa.model import Model

## Build a base network from TZA-PyPSA-ASEAN

In [ ]:
network = (
    Model.load_model(
        'ASEAN', 
        frequency = '1h',
        select_nodes = ['MYSPE'], 
        years = [2030],
        backstop = False,
        set_global_constraints = False,
    )
)

## Define function to build a CFE network

In [ ]:
def PrepareNetworkForCFE(network, buses_with_ci_load):
    '''
    
    In this function, we take a PyPSA network and create a virtual subsystem representing 
        a C&I asset. Here, we loop through a set of buses in our brownfield network where
        we want to add a C&I load. We create a subsystem attached to each define bus, as 
        we do in the demo script / schematic.

    '''

    clean_carriers = ['photovoltaic-unspecified']
    clean_storages = ['lithium-ion']

    n = network.copy()

    # loop and add buses
    for bus in buses_with_ci_load:

        # define C&I bus
        ci_bus = f'{bus} C&I Grid'
        ci_storage = f'{bus} C&I Storage'

        # add bus for C&I system
        n.add(
            "Bus",
            ci_bus,
            x = n.buses.x.iloc[0] + 1,
            y = n.buses.y.iloc[0] + 1,       
        )

        # add bus for C&I storage
        n.add(
            "Bus",
            ci_storage,
            x = n.buses.x.iloc[0] - 1,
            y = n.buses.y.iloc[0] - 1,
        )

        # add C&I load
        n.add(
            "Load",
            f'{bus} C&I Load',
            bus = ci_bus,
            p_set = n.loads_t.p_set[bus] * 0.1,
        )

        # add clean generator for PPA
        n.add(
            "Generator",
            f'{bus} PPA Generation',
            bus = ci_bus,
            carrier = 'C&I Clean',
            p_nom = 0,
            p_nom_extendable = True,
            capital_cost = 1e12,
            marginal_cost = 1e3,
            p_max_pu = n.generators_t.p_max_pu.filter(regex='photo').values.flatten(),
        )

        # add clean storage for PPA
        n.add(
            "StorageUnit",
            f"{bus} PPA Storage",
            bus=ci_storage,
            p_nom=0,
            p_nom_extendable=True,
            cyclic_state_of_charge=True,
            capital_cost=1e9,
            marginal_cost=0,
            max_hours=4,
            efficiency_store=0.9,
            efficiency_dispatch=0.9,
        )

        # add links to connect Bus <-> C&I System
        n.add(
            "Link",
            f"{bus} Grid Imports",
            bus0=bus, 
            bus1=ci_bus, 
            p_nom=1e12,
            marginal_cost=1,
        )

        n.add(
            "Link",
            f"{bus} Grid Exports",
            bus0=ci_bus, 
            bus1=bus, 
            p_nom=1e12,
            marginal_cost=1,
        )

        # add links to connect C&I System <-> Storage
        n.add(
            "Link",
            f"{bus} StorageCharge",
            bus0=ci_bus, 
            bus1=ci_storage, 
            p_nom=0,
            p_nom_extendable=True,
            marginal_cost=1,
        )

        n.add(
            "Link",
            f"{bus} StorageDischarge",
            bus0=ci_storage, 
            bus1=ci_bus, 
            p_nom=0,
            p_nom_extendable=True,
            marginal_cost=1,
        )

    return n

## Run a brownfield network simulation

In [ ]:
brownfield = network.copy()

# prep for CFE (though nothing happens on C&I system in brownfield)
brownfield = PrepareNetworkForCFE(network, buses_with_ci_load=['MYSPE'])

# optimise
brownfield.optimize(solver_name='gurobi')

## Run a cfe simulation

In [ ]:
n

## Post-processing

In [ ]:
brownfield.statistics.energy_balance() / 1e5

In [ ]:
cfe.statistics.energy_balance()  / 1e5

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_dispatch(n, ci_label='C&I', timesteps=24):

    gen = n.generators_t.p.filter(regex=ci_label).sum(axis=1).iloc[0:timesteps].to_frame('PPA')

    if not n.storage_units.empty:
        sto = n.storage_units_t.p.filter(regex=ci_label).sum(axis=1).iloc[0:timesteps].to_frame('Storage')
        
    imports = n.links_t.p0.filter(regex='Import').sum(axis=1).iloc[0:timesteps].to_frame('Import')
    exports = n.links_t.p0.filter(regex='Export').sum(axis=1).iloc[0:timesteps].to_frame('Export').mul(-1)

    p_by_carrier = pd.concat([gen, sto, imports, exports], axis=1)

    fig, ax = plt.subplots(figsize=(6, 3))

    color_dict = {
        'PPA' : 'teal',
        'Storage' : 'peachpuff',
        'Import' : 'coral',
        'Export' : 'peru',
    }

    color = (
        p_by_carrier
        .columns
        .map(color_dict)
    )

    p_by_carrier.where(p_by_carrier > 0).iloc[0:timesteps].plot.area(
        ax=ax,
        linewidth=0,
        color=color,
    )

    charge = p_by_carrier.where(p_by_carrier < 0).dropna(how="all", axis=1)

    if not charge.empty:
        charge.plot.area(
            ax=ax,
            linewidth=0,
            color=charge.columns.map(color_dict),
        )

    n.loads_t.p_set.iloc[0:timesteps].filter(regex='C&I Load').plot(ax=ax, c="k")

    ax.set_ylim([-1e4,1e4])


plot_dispatch(cfe, ci_label='PPA', timesteps=24*4)

In [10]:
components = ['generators', 'links', 'storage_units']
for c in components:
    print( getattr(network, c)['p_nom'] )

Generator
MYSPE-bioenergy-unspecified-ext-2030            0.00
MYSPE-combined-cycle-ext-2030               12669.20
MYSPE-coal-subcritical-ext-2030              8200.00
MYSPE-coal-ultrasupercritical-ext-2030       4000.00
MYSPE-hydro-unspecified-ext-2030             2536.10
MYSPE-open-cycle-gas-turbine-ext-2030         834.00
MYSPE-photovoltaic-unspecified-ext-2030      1445.42
MYSPE-waste-ext-2030                            0.00
MYSPE-wind-offshore-unspecified-ext-2030        0.00
MYSPE-wind-onshore-ext-2030                     0.00
Name: p_nom, dtype: float64
Series([], Name: p_nom, dtype: float64)
StorageUnit
MYSPE-lithium-ion-ext-2030    0.0
Name: p_nom, dtype: float64


In [18]:
n = pypsa.Network('/Users/aman/Library/CloudStorage/GoogleDrive-aman.m@transitionzero.org/My Drive/tz-pypsa-workspace/tza-google-cfe/run/N_BROWNFIELD.nc')

INFO:pypsa.io:Imported network N_BROWNFIELD.nc has buses, carriers, generators, links, loads, storage_units


In [27]:
n.links

,bus0,bus1,carrier,p_nom_extendable,capital_cost,marginal_cost,p_nom_opt,type,efficiency,build_year,...,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
Link,,,,,,,,,,,,,,,,,,,,,
MYSPE C&I Grid Imports,MYSPE,MYSPE C&I Grid,AC,True,0.01,0.01,2640.36638,,1.0,0,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
MYSPE C&I Grid Exports,MYSPE C&I Grid,MYSPE,AC,False,0.01,0.01,0.00000,,1.0,0,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
MYSPE C&I Storage Charge,MYSPE C&I Grid,MYSPE C&I Storage,AC,False,0.01,0.01,0.00000,,1.0,0,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
MYSPE C&I Storage Discharge,MYSPE C&I Grid,MYSPE C&I Storage,AC,False,0.01,0.01,0.00000,,1.0,0,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
